In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)
sess = tf.InteractiveSession()

In [2]:

# One hot encoding for each char in 'hello'
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [3]:

with tf.variable_scope('one_cell') as scope:
    # One cell RNN input_dim (4) -> output_dim (2)
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    print(cell.output_size, cell.state_size)

    x_data = np.array([[h]], dtype=np.float32) # x_data = [[[1,0,0,0]]]
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

W0922 02:17:07.862104 10828 deprecation.py:506] From W:\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0922 02:17:07.870098 10828 deprecation.py:323] From <ipython-input-3-9e94a5d57760>:10: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
array([[[0.54985994, 0.13801265]]], dtype=float32)


In [13]:
with tf.variable_scope('two_sequances') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5
    hidden_size = 4
    cell = tf.keras.layers.SimpleRNNCell(units=hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype=np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    print("random weight")
    pp.pprint(outputs.eval())
    

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
random weight
array([[[ 0.4780785 , -0.53581154,  0.63159   , -0.20868747],
        [-0.37248346,  0.8466291 ,  0.8184807 ,  0.35754973],
        [ 0.9223282 ,  0.05269922, -0.6390722 ,  0.5481024 ],
        [ 0.43175104, -0.76569724, -0.7997321 ,  0.9640457 ],
        [ 0.47709706, -0.6623714 ,  0.3652202 ,  0.86779344]]],
      dtype=float32)


In [5]:
with tf.variable_scope('3_batches') as scope:
    # One cell RNN input_dim (4) -> output_dim (2). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 4
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

W0922 02:17:11.626789 10828 deprecation.py:323] From <ipython-input-5-71f3444a061b>:10: LSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0922 02:17:11.759706 10828 deprecation.py:506] From W:\Anaconda3\lib\site-packages\tensorflow\python\ops\rnn_cell_impl.py:961: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.11812446, -0.02722695, -0.12213697,  0.01296182],
        [ 0.04207598, -0.08073659, -0.08601348, -0.07155815],
        [ 0.05565977, -0.05115978,  0.07826982, -0.06628758],
        [ 0.05479522,  0.00615801,  0.15904328, -0.05869433],
        [ 0.05035336,  0.00745668,  0.26079223, -0.1269741 ]],

       [[ 0.09703065, -0.05189857, -0.01488282, -0.07521112],
        [ 0.08147444, -0.04996339,  0.14482333, -0.14507683],
        [ 0.05054832,  0.00692039,  0.17292832, -0.10059192],
        [ 0.04776085,  0.05284648,  0.21552378, -0.07956108],
        [ 0.04479758,  0.0863

In [6]:
with tf.variable_scope('3_batches_dynamic_length') as scope:
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch 3
    # 3 batches 'hello', 'eolll', 'lleel'
    # more effective way to learn than 'two_sequances'
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    hidden_size = 4
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, sequence_length=[5,3,4], dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

W0922 02:17:30.750156 10828 deprecation.py:323] From W:\Anaconda3\lib\site-packages\tensorflow\python\ops\rnn.py:244: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.02620494,  0.10164541,  0.02815075,  0.04961136],
        [-0.07109021,  0.17554677,  0.12538923,  0.10818171],
        [-0.12590589,  0.02051727,  0.10917158,  0.20803843],
        [-0.15294129, -0.09503551,  0.13059798,  0.25720114],
        [-0.11531893, -0.04498752,  0.19436504,  0.04040595]],

       [[-0.08544837,  0.12009084,  0.09972579,  0.06048245],
        [-0.09176629,  0.15011188,  0.1337932 , -0.07111263],
        [-0.16170236, -0.0030831 ,  0.11365655,  0.08169305],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.    

In [7]:
with tf.variable_scope('initial_state') as scope:
    batch_size = 3
    x_data = np.array([[h, e, l, l, o],
                      [e, o, l, l, l],
                      [l, l, e, e, l]], dtype=np.float32)
    pp.pprint(x_data)
    
    # One cell RNN input_dim (4) -> output_dim (5). sequence: 5, batch: 3
    hidden_size=4
    cell = tf.nn.rnn_cell.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                         initial_state=initial_state, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.10253008, -0.1578676 ,  0.10426318, -0.03880213],
        [ 0.0178168 , -0.15115373,  0.21053277, -0.15495834],
        [ 0.11915527, -0.11595725,  0.17822985, -0.06189169],
        [ 0.17642874, -0.06937148,  0.1705166 , -0.01912814],
        [ 0.17781428,  0.00726382,  0.05526971, -0.0089156 ]],

       [[-0.04347115, -0.07335197,  0.0812296 , -0.13306615],
        [ 0.05810375,  0.00281493, -0.04749247, -0.07705914],
        [ 0.14259636,  0.02332217,  0.00903249, -0.02747288],
        [ 0.19163826,  0.02843427,  0.04908187,  0.01311253],
        [ 0.2216192 ,  0.0257